# 설정
* 외부데이터(기온, 기압, 습도)
    * 'month', 'weekday', '시간' 특성으로 기온 예측
    * 기온 예측할 때 사용한 데이터에 예측한 기온 포함한 데이터로 기압 예측
    * 기압 예측할 때 사용한 데이터에 예측한 기압 포함한 데이터로 습도 예측
    * 기온 : cat, lgbm, xgb 예측 평균
    * 기압, 습도 : et, dt, rf 예측 평균
* 가스공급량 데이터 : 이상치 제거
* pycaret 동일 setup 파라미터
    * robust, transformation
* sub48_ver3.9(0.1024290902)
    * 공급량 예측(6개) : 'month', 'weekday', '시간', '구분', '기온', '기압'
    * 모델 : catboost, lightgbm 예측 평균
* pycaret_robust_transformation_blender.csv(0.1021660476)
    * 공급량 예측(7개) : 'month', 'weekday', '시간', '구분', '기온', '기압', '습도'
    * 모델 : catboost, lightgbm, xgboost. 3모델 blend 예측

In [1]:
from pycaret.regression import *
import pandas as pd

In [ ]:
# team_sub_ver0.4.csv
# 자체 점수 : 0.00640, 0.99871
# 제출 점수 : 0.10208606(2021-12-10 10:48:55)

# 각 팀원 top1 파일 가져오기

In [2]:
# sub48_ver3.9(0.1024290902)
sub01 = pd.read_csv('../07_결과_출력/sub48_ver3.9.csv')
# pycaret_robust_transformation_blender.csv(0.1021660476)
sub02 = pd.read_csv('../07_결과_출력/pycaret_robust_transformation_blender.csv')

In [3]:
sub01.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,1989.123559
1,2019-01-01 02 A,1768.301622
2,2019-01-01 03 A,1739.008780
3,2019-01-01 04 A,1728.737200
4,2019-01-01 05 A,1905.308220


In [5]:
sub01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
dtypes: float64(1), object(1)
memory usage: 236.4+ KB


In [4]:
sub02.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,1949.279833
1,2019-01-01 02 A,1833.490265
2,2019-01-01 03 A,1744.566888
3,2019-01-01 04 A,1756.344474
4,2019-01-01 05 A,1889.023081


In [6]:
sub02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
dtypes: float64(1), object(1)
memory usage: 236.4+ KB


## 두 제출 파일 결과 하나로 합치기

In [8]:
sub02 = sub02.drop('일자|시간|구분', axis=1)
sub02.head()

,공급량
0,1949.279833
1,1833.490265
2,1744.566888
3,1756.344474
4,1889.023081


In [9]:
df = pd.concat([sub01, sub02], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   일자|시간|구분  15120 non-null  object 
 1   공급량       15120 non-null  float64
 2   공급량       15120 non-null  float64
dtypes: float64(2), object(1)
memory usage: 354.5+ KB


In [10]:
df.columns = ['일자|시간|구분', 'sub01', 'sub02']
df.head()

,일자|시간|구분,sub01,sub02
0,2019-01-01 01 A,1989.123559,1949.279833
1,2019-01-01 02 A,1768.301622,1833.490265
2,2019-01-01 03 A,1739.008780,1744.566888
3,2019-01-01 04 A,1728.737200,1756.344474
4,2019-01-01 05 A,1905.308220,1889.023081


In [11]:
df['공급량'] = (df['sub01'] + df['sub02']) / 2
df.head()

,일자|시간|구분,sub01,sub02,공급량
0,2019-01-01 01 A,1989.123559,1949.279833,1969.201696
1,2019-01-01 02 A,1768.301622,1833.490265,1800.895943
2,2019-01-01 03 A,1739.008780,1744.566888,1741.787834
3,2019-01-01 04 A,1728.737200,1756.344474,1742.540837
4,2019-01-01 05 A,1905.308220,1889.023081,1897.165651


In [12]:
df = df.drop(['sub01', 'sub02'], axis=1)
df.head()

,일자|시간|구분,공급량
0,2019-01-01 01 A,1969.201696
1,2019-01-01 02 A,1800.895943
2,2019-01-01 03 A,1741.787834
3,2019-01-01 04 A,1742.540837
4,2019-01-01 05 A,1897.165651


# 제출 파일 만들기

In [15]:
df.to_csv('team_sub_ver0.4.csv', index=False)